In [1]:
text = """✅ Elasticsearch security features have been automatically configured!
✅ Authentication is enabled and cluster connections are encrypted.

ℹ️  Password for the elastic user (reset with `bin/elasticsearch-reset-password -u elastic`):
  7l5NNDht_jsMzzExtWUo

ℹ️  HTTP CA certificate SHA-256 fingerprint:
  0fb63522175c33c1842b5b5678986c10fa78b04ae264032beda52c2578683a8e

ℹ️  Configure Kibana to use this cluster:
• Run Kibana and click the configuration link in the terminal when Kibana starts.
• Copy the following enrollment token and paste it into Kibana in your browser (valid for the next 30 minutes):
  eyJ2ZXIiOiI4LjEzLjIiLCJhZHIiOlsiMTcyLjE4LjAuMjo5MjAwIl0sImZnciI6IjBmYjYzNTIyMTc1YzMzYzE4NDJiNWI1Njc4OTg2YzEwZmE3OGIwNGFlMjY0MDMyYmVkYTUyYzI1Nzg2ODNhOGUiLCJrZXkiOiJub00xNjQ0Qm8zWXNzYTNvV1ZpQTpXNDAwWUVMeVRwbUtYcEFpeUNfUHZ3In0=

ℹ️ Configure other nodes to join this cluster:
• Copy the following enrollment token and start new Elasticsearch nodes with `bin/elasticsearch --enrollment-token <token>` (valid for the next 30 minutes):
  eyJ2ZXIiOiI4LjEzLjIiLCJhZHIiOlsiMTcyLjE4LjAuMjo5MjAwIl0sImZnciI6IjBmYjYzNTIyMTc1YzMzYzE4NDJiNWI1Njc4OTg2YzEwZmE3OGIwNGFlMjY0MDMyYmVkYTUyYzI1Nzg2ODNhOGUiLCJrZXkiOiJuNE0xNjQ0Qm8zWXNzYTNvV1ZpQTowbHJSX25mYVNSYThkNFBLaGpMLXB3In0=

  If you're running in Docker, copy the enrollment token and run:
  `docker run -e "ENROLLMENT_TOKEN=<token>" docker.elastic.co/elasticsearch/elasticsearch:8.13.2`"""

In [2]:
key = "MHJIdjZJNEJUMHZualkxWFpKV2k6aUVRR0w1R3dSMHFrczFqVnZOcnR4QQ=="
address = "https://localhost:9200"
password = "7l5NNDht_jsMzzExtWUo"
fingerprint = "0fb63522175c33c1842b5b5678986c10fa78b04ae264032beda52c2578683a8e"
enrollment = "eyJ2ZXIiOiI4LjEzLjIiLCJhZHIiOlsiMTcyLjE4LjAuMzo5MjAwIl0sImZnciI6IjdhYzJmN2U2MGRmOWE0ZTMwMGQwMDAyODc1MWNiMjhjZWVmOTQyMmE2ZGM3OWQ4MWRmZjMwNWEyY2FkNTcyNjEiLCJrZXkiOiJsWjhjNjQ0QklqQkg5UVVZREQ4NDpURndmNng0OVFHU1dCeHpMd1FXNGVRIn0="

In [55]:
from elasticsearch import Elasticsearch
from datetime import datetime
import sys
import os

In [56]:
client = Elasticsearch(address, ssl_assert_fingerprint=fingerprint, basic_auth=("elastic", password))
client.info()

ObjectApiResponse({'name': '2621eaa976cd', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'DaLmFfnHQNKan2DPS_62NQ', 'version': {'number': '8.13.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '16cc90cd2d08a3147ce02b07e50894bc060a4cbf', 'build_date': '2024-04-05T14:45:26.420424304Z', 'build_snapshot': False, 'lucene_version': '9.10.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

## Test Run

In [57]:
test = {
    'author': 'author_name',
    'text': 'Interesting content...',
    'timestamp': datetime.now(),
}

In [58]:
resp = client.index(index="test-index", id=1, document=test)
print(resp['result'])

created


In [7]:
resp =  client.get(index="test-index", id=1)
print(resp['_source'])

{'author': 'author_name', 'text': 'Interesting content...', 'timestamp': '2024-04-17T11:55:15.550382'}


In [8]:
client.indices.refresh(index="test-index")

ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}})

In [9]:
resp = client.search(index="test-index", query={"match_all": {}})
print(f"Got {resp['hits']['total']['value']} hits. Hits:")
for hit in resp['hits']['hits']:
    print("%(timestamp)s %(author)s: %(text)s" % hit["_source"])

Got 1 hits. Hits:
2024-04-17T11:55:15.550382 author_name: Interesting content...


In [10]:
doc = {
    'author': 'Bob',
    'text': 'The quick brown fox jumps over the lazy dog',
    'timestamp': datetime.now(),
}

resp = client.update(index="test-index", id=1, doc=doc)
print(resp['result'])

updated


In [11]:
client.delete(index="test-index", id=1)

ObjectApiResponse({'_index': 'test-index', '_id': '1', '_version': 3, 'result': 'deleted', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1})

In [60]:
client.indices.delete(index="test-index")

ObjectApiResponse({'acknowledged': True})

## Test indices

In [14]:
try:
    client.indices.create(index="document_index")
except Exception as e:
    print(e)

BadRequestError(400, 'resource_already_exists_exception', 'index [document_index/UpQJ5PAAROajOEW6Nz-4Fg] already exists')


In [32]:
count = 0
for root, dirs, files in os.walk("./davisWiki"):
    for file in files:
        with open(os.path.join(root, file), "r") as f:
            try:
                title = file
                text = f.read()
                doc = {
                    "title": title,
                    "text": text,
                    "timestamp": datetime.now()
                }
                count += 1
                client.index(index="document_index", id=count, document=doc)
                if count % 5000 == 0:
                    print(f"Processing document {count}")
                
            except Exception as e:
                print(e)
                print("title: ", title)
                break

print(f"Processing document {count}") 
print("done")
                
            

Processing document 5000
Processing document 10000
Processing document 15000
Processing document 17478
done


In [59]:
query = "zombie attack"

resp = client.search(index="document_index", body={"query": {
    "match": {
        "text": query
}}, "size": 10, "explain": True})

count = 0 
print(f"Got {resp['hits']['total']['value']} hits. Hits:")
for hit in resp['hits']['hits']:
    print("%(title)s" % hit["_source"])
    

Got 248 hits. Hits:
Kearney_Hall.f
Zombie_Walk.f
Measure_Z.f
Spirit_Halloween.f
Zombie_Attack_Response_Guide.f
Scream.f
Disasters.f
Biological_Disasters.f
JasonRifkind.f
Zombies_Reclaim_the_Streets.f
